In [ ]:
from tqdm import tqdm
import numpy as np

# Constants
SBOX =  np.array(
        [0x63, 0x7c, 0x77, 0x7b, 0xf2, 0x6b, 0x6f, 0xc5, 0x30, 0x01, 0x67,
        0x2b, 0xfe, 0xd7, 0xab, 0x76, 0xca, 0x82, 0xc9, 0x7d, 0xfa, 0x59,
        0x47, 0xf0, 0xad, 0xd4, 0xa2, 0xaf, 0x9c, 0xa4, 0x72, 0xc0, 0xb7,
        0xfd, 0x93, 0x26, 0x36, 0x3f, 0xf7, 0xcc, 0x34, 0xa5, 0xe5, 0xf1,
        0x71, 0xd8, 0x31, 0x15, 0x04, 0xc7, 0x23, 0xc3, 0x18, 0x96, 0x05,
        0x9a, 0x07, 0x12, 0x80, 0xe2, 0xeb, 0x27, 0xb2, 0x75, 0x09, 0x83,
        0x2c, 0x1a, 0x1b, 0x6e, 0x5a, 0xa0, 0x52, 0x3b, 0xd6, 0xb3, 0x29,
        0xe3, 0x2f, 0x84, 0x53, 0xd1, 0x00, 0xed, 0x20, 0xfc, 0xb1, 0x5b,
        0x6a, 0xcb, 0xbe, 0x39, 0x4a, 0x4c, 0x58, 0xcf, 0xd0, 0xef, 0xaa,
        0xfb, 0x43, 0x4d, 0x33, 0x85, 0x45, 0xf9, 0x02, 0x7f, 0x50, 0x3c,
        0x9f, 0xa8, 0x51, 0xa3, 0x40, 0x8f, 0x92, 0x9d, 0x38, 0xf5, 0xbc,
        0xb6, 0xda, 0x21, 0x10, 0xff, 0xf3, 0xd2, 0xcd, 0x0c, 0x13, 0xec,
        0x5f, 0x97, 0x44, 0x17, 0xc4, 0xa7, 0x7e, 0x3d, 0x64, 0x5d, 0x19,
        0x73, 0x60, 0x81, 0x4f, 0xdc, 0x22, 0x2a, 0x90, 0x88, 0x46, 0xee,
        0xb8, 0x14, 0xde, 0x5e, 0x0b, 0xdb, 0xe0, 0x32, 0x3a, 0x0a, 0x49,
        0x06, 0x24, 0x5c, 0xc2, 0xd3, 0xac, 0x62, 0x91, 0x95, 0xe4, 0x79,
        0xe7, 0xc8, 0x37, 0x6d, 0x8d, 0xd5, 0x4e, 0xa9, 0x6c, 0x56, 0xf4,
        0xea, 0x65, 0x7a, 0xae, 0x08, 0xba, 0x78, 0x25, 0x2e, 0x1c, 0xa6,
        0xb4, 0xc6, 0xe8, 0xdd, 0x74, 0x1f, 0x4b, 0xbd, 0x8b, 0x8a, 0x70,
        0x3e, 0xb5, 0x66, 0x48, 0x03, 0xf6, 0x0e, 0x61, 0x35, 0x57, 0xb9,
        0x86, 0xc1, 0x1d, 0x9e, 0xe1, 0xf8, 0x98, 0x11, 0x69, 0xd9, 0x8e,
        0x94, 0x9b, 0x1e, 0x87, 0xe9, 0xce, 0x55, 0x28, 0xdf, 0x8c, 0xa1,
        0x89, 0x0d, 0xbf, 0xe6, 0x42, 0x68, 0x41, 0x99, 0x2d, 0x0f, 0xb0,
        0x54, 0xbb, 0x16])

In [ ]:
def rank(predictions, metadata, real_key, traces_idxs, last_key_bytes_proba, target_byte, simulated_key):
	# Compute the rank
	if len(last_key_bytes_proba) == 0:
		# If this is the first rank we compute, initialize all the estimates to zero
		key_bytes_proba = np.zeros(256)
	else:
		# This is not the first rank we compute: we optimize things by using the
		# previous computations to save time!
		key_bytes_proba = last_key_bytes_proba

	for t_idx in traces_idxs:
		# Go back from the class to the key byte. '2' is the index of the byte (third byte) of interest.
		plaintext = metadata[t_idx]['plaintext'][target_byte]
		key = metadata[t_idx]['key'][target_byte]
		for key_guess in range(0, 256):
			# Our candidate key byte probability is the sum of the predictions logs
			if (simulated_key!=1):
				proba = predictions[t_idx][SBOX[plaintext ^ key_guess]]
			else:
				proba = predictions[t_idx][SBOX[plaintext ^ key ^ key_guess]]
			if proba != 0:
				key_bytes_proba[key_guess] += np.log(proba)
			else:
				# We do not want an -inf here, put a very small epsilon
				# that correspondis to a power of our min non zero proba
				min_proba_predictions = predictions[t_idx][np.array(predictions[t_idx]) != 0]
				if len(min_proba_predictions) == 0:
					print("Error: got a prediction with only zeroes ... this should not happen!")
					sys.exit(-1)
				min_proba = min(min_proba_predictions)
				key_bytes_proba[key_guess] += np.log(min_proba**2)
	# Now we find where our real key candidate lies in the estimation.
	# We do this by sorting our estimates and find the rank in the sorted array.
	sorted_proba = np.array(list(map(lambda a : key_bytes_proba[a], key_bytes_proba.argsort()[::-1])))
	real_key_rank = np.where(sorted_proba == key_bytes_proba[real_key])[0][0]
	return (real_key_rank, key_bytes_proba)

In [ ]:
def run_rank_trials(predictions, metadata, n_traces_per_trial=1000, n_trials=10, rank_step=10, target_byte=2, simulated_key=0):
    real_key = metadata[0]['key'][target_byte] if not simulated_key else 0

    max_trace_idx = predictions.shape[0]
    steps = list(range(rank_step, n_traces_per_trial + 1, rank_step))

    all_ranks = []

    for trial in tqdm(range(n_trials), desc="Trials"):
        trace_indices = np.random.choice(np.arange(max_trace_idx), size=n_traces_per_trial, replace=False)
        trial_ranks = []
        key_bytes_proba = []

        for i in tqdm(steps, leave=False, desc=f"Rank steps (Trial {trial+1})"):
            selected = trace_indices[:i]
            rank_val, key_bytes_proba = rank(predictions, metadata, real_key, selected, key_bytes_proba, target_byte, simulated_key)
            trial_ranks.append(rank_val)

        all_ranks.append(trial_ranks)

    return steps, np.array(all_ranks)

In [ ]:
def plot_mean_rank(steps, ranks_matrix):
    import matplotlib.pyplot as plt
    mean_ranks = np.mean(ranks_matrix, axis=0)
    std_ranks = np.std(ranks_matrix, axis=0)

    plt.plot(steps, mean_ranks, label='Mean Rank')
    plt.fill_between(steps, mean_ranks - std_ranks, mean_ranks + std_ranks, alpha=0.3, label='±1 std dev')
    plt.xlabel("Number of traces")
    plt.ylabel("Rank of correct key")
    plt.title("Mean key rank across trials")
    plt.grid(True)
    plt.legend()
    plt.show()